# 9장 다시 피처로: 학술 논문 추천 시스템 구축
- item-based collaborative filtering

## 9장에서 사용할 데이터 생성

In [ ]:
import pandas as pd

In [ ]:
file_path = 'data/mag_papers/'
model_df = pd.read_json(file_path + "mag_papers_0.txt", lines=True)
model_df.shape

In [ ]:
# 예제에서 2만 건의 데이터만 사용

df20000 = model_df[:20000, :]
df20000.shape

In [ ]:
df20000.to_json(file_path + "mag_subset20k.txt", orient='records', lines=True)

### __첫번째 단계: 데이터 가져오기, 정제하기, 피처 파싱하기__
- 모든 과학 실험과 맟나가지로 이 작업도 가설로 시작할 것이다. __이 경우에는 거의 같은 시기에 비슷한 연구 분야에서 출간된 논문이 사용자에게 가장 유용할 것이라고 가정한다.__ 그래서 전체 데이터셋의 하위 샘플에서 이와 관련된 필드를 파싱하는 단순한 접근법을 사용할 것이다. 간단한 희소 배열을 생성한 다음에 항목 기반 협업 필터를 전체 항목 배열에 적용해 좋은 결과가 나오는지 확인해보자.

항목 기반 협업 필터는 항목을 비교하는 유사도 점수에 좌우된다. 이 경우에는 코사인 유사도가 두 개의 0이 아닌 벡터를 비교하기 위해 사용된다. 실제로 다음 예제는 양수의 공간에서 코사인 유사도를 보완하는 코사인 거리를 사용한다.

### Example 9-1. Import + filter data
- 단순 접근법

In [ ]:
model_df = pd.read_json('data/mag_papers/mag_subset20K.txt', lines=True)
model_df.shape

In [ ]:
model_df.columns

In [ ]:
# 영어가 아닌 논문 제외
# 제목이 중복인 것 제외

model_df = model_df[model_df.lang == 'en'].drop_duplicates(subset='title', keep='first')

# abstract, authors, fos, keytwords, year, title 컬럼만 사용
model_df = model_df.drop(['doc_type', 
                          'doi', 'id', 
                          'issue', 'lang', 
                          'n_citation', 
                          'page_end', 
                          'page_start', 
                          'publisher', 
                          'references',
                          'url', 
                          'venue', 
                          'volume'], axis=1)

In [ ]:
model_df.shape

In [ ]:
model_df.head(2)

### Example 9-2. Collaborative filtering stage 1: Build item feature matrix

In [ ]:
from sys import getsizeof
import random

In [ ]:
# 연구분야
unique_fos = sorted({feature 
                     for paper_row in model_df.fos.fillna('0')
                     for featrue in paper_row})

# 출간년도
unique_year = sorted(model_df['year'].astype('str').unique())

print("unique_fos  :", len(unique_fos))
print("unique_year :", len(unique_year))
print("total       :", len(unique_fos + unique_year))

In [ ]:
# 연구분야가 null이 아닌 데이터 수
model_df.shape[0] - pd.isnull(model_df['fos']).sum()

In [ ]:
[unique_fos[i] for i in sorted(random.sample(range(len(unique_fos)), 15)) ]

In [ ]:
def feature_array(x, unique_array):
    row_dict = {}
    for i in x.index:
        var_dict = {}
        
        for j in range(len(unique_array)):
            if type(x[i]) is list:
                if unique_array[j] in x[i]:
                    var_dict.update({unique_array[j]: 1})
                else:
                    var_dict.update({unique_array[j]: 0})
            else:
                if unique_array[j] == str(x[i]):
                    var_dict.update({unique_array[j]: 1})
                else:
                    var_dict.update({unique_array[j]: 0})
        
        row_dict.update({i : var_dict})
    
    feature_df = pd.DataFrame.from_dict(row_dict, dtype='str').T
    
    return feature_df

In [ ]:
%time year_features = feature_array(model_df['year'], unique_year)

In [ ]:
%time fos_features = feature_array(model_df['fos'], unique_fos)

print("Size of fos feature array: ", getsizeof(fos_features))

In [ ]:
print(year_features.shape)
print(fos_feature.shape)

In [ ]:
year_features.shape[1] + fos_features.shape[1]

In [ ]:
# 10399 X 7760 array
%time first_features = fos_features.join(year_features).T

first_size = getsizeof(first_features)
print("Size of first feature array: ", first_size)

In [ ]:
first_features.shape

In [ ]:
first_features.head()

### Example 9-3. Collaborative filtering stage 2: Search for similar items

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
def item_collab_filter(features_df):
    item_similarities = pd.DataFrame(index=features_df.columns, columns=features_df.columns)
    
    for i in features_df.columns:
        for j in featrues_df.columns:
            item_similarities.loc[i][j] = 1 - cosine(features_df[i].astype('float'), features_df[j].astype('float'))
    
    return item_similarities

In [ ]:
%time first_items = item_collab_filter(first_features.loc[:, 0:1000])

### Example 9-4. Heatmap of paper recommendations

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# 그림 9-3
sns.set()
ax = sns.heatmap(first_items.fillna(0),
                vmin=0, vmat=1,
                cmap='Y1GnBu',
                xticklabels=250, yticklabels=250)
ax.tick_params(labelsiz=12)

### Example 9-5. Item-based collaborative filtering recommendations

In [ ]:
def paper_recommender(paper_index, items_df):
    print("Based on the paper: \nindex = ", paper_index)
    print(model_df.iloc[paper_index])
    top_results = item_df.loc[paper_index].sort_values(ascending=False).head(4)
    print("\nTop3 results: ")
    order = 1
    
    for i in top_results.index.tolist()[-3:]:
        print(order, '. Paper index = ', i)
        print("Similarity score: ", top_results[i])
        print(model_df.iloc[i], "\n")
        if order < 5: order += 1

In [ ]:
paper_recommender(2, first_items)

## __두번째 단계: 피처 엔지니어링과 더 똑똑한 모델__
- 커다란 희소 배열을 생성하고 그것을 강제적으로 필터에 밀어너는 첫 번째 접근법은 다양한 방식으로 개선될 수 있다. 다음 단계에서는 처음에 사용한 두 개의 피처에 더 나은 기법을 적용하고 더 빠르게 반복 처리를 하도록 항목 기반 협업 필터를 수정하는 데 중점을 둘 것이다. 우선 우리의 가설에 사용한 두 개의 변수에 몇 가지 피처 엔지니어링 기법을 적용해보자. 앞서 개발한 피처를 더 자세히 살펴보면, 각 변수의 유형에 맞춰 추천 시스템을 위한 '더 나은'피처롤 변환시킬 방법을 선택할 수 있다.

### Example 9-6. Fixed-width binning + dummy coding(part 1)
- 고정 폭 비닝 + 더미 코딩(파트1)

In [ ]:
model_df['year'].tail()

In [ ]:
print("Year spread: ", model_df['year'].min(), " - ", model_df['year'].max())
print("Quantile spread:\n", model_df['year'].quantile([0.25, 0.5, 0.75]))

In [ ]:
# 그림 9-5. year의 분포 확인
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(7, 5))
model_df['year'].hist(ax=ax, bins=model_df['year'].max() - model_df['year'].min())
ax.tick_params(labelsize=12)
ax.set_xlabel("Year Count", fontsize=12)
ax.set_ylabel("Occurrence", fontsize=12)

### Example 9-7. Fixed-width binning + dummy coding (part 2)

In [ ]:
# bin은 데이터의 수가 아니라 변수의 범위를 기준으로 설정한다.
model_df['year'].max() - model_df['year'].min()

In [ ]:
# year 피처를 10년 단위로 비닝
bins = int(round((model_df['year'].max() = model_df['year'].min()) / 10))

temp_df = pd.DataFrame(index = model_df.index)
temp_df['yearBinned'] = pd.cut(model_df['year'].tolist(), bins, precision = 0)

In [ ]:
# year 피처를 10년 단위로 비닝함으로써 피처 공간을 156에서 19로 줄인다
print("We have reduced from", len(mode_df['year'].unique()), 
     "to", len(temp_df['yearBinned'].values.unique()), "features representing the year.")

In [ ]:
X_yrs = pd.get_dummies(temp_df['yearBinned'])
X_yrs.head()

In [ ]:
X_yrs.columns.categories

In [ ]:
# 그림 9-6. 비닝한 year의 분포 확인
sns.set_style('white')
fig, ax = plt.subplots(figsize=(7, 5))
X_yrs.sum().plot.bar(ax = ax)
ax.tick_params(labelsize=12)
ax.set_xlabel("Binned Years", fontsize=12)
ax.set_ylabel("Counts", fontsize=12)

### Example 9-8. Converting bag-of-phrases pd.Series to NumPy sparse array
- Pandas 시리즈인 bag-of-pharses를 NumPy 희소 배열로 변환

In [ ]:
X_fos = fos_features.values

In [ ]:
# 각 객체의 크기를 보면 나중에 어떤 차이를 만들게 될지 예상할 수 있다.
print('Our pandas Series, in bytes: ', getsizeof(fos_features))
print('Our hashed numpy array, in bytes: ', getsizeof(X_fos))

### Example 9-9. Collaborative filtering stages 1 + 2: Build item feature matrix, search for similar items
- 항목 피처 행렬 생성, 유사 항목 검색

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklear.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DicVectorizer

In [ ]:
X_yrs.shape[1] + X_fos.shape[1]

In [ ]:
# 10399 x 7623 array

%time second_features = np.append(X_fos, X_yrs, axis=1)

second_size = getsizeof(second_features)
print("Size of second feature array, in bytes: ", scond_size)

In [ ]:
print("The power of feature engineering saves us, in bytes: ", getsizeof(fos_features) - second_size)

In [ ]:
def piped_collab_filter(features_matrix, index, top_n):
    item_similarities = 1 - cosine_similarity(features_matrix[index:index+1], features_matrix).flatten()
    related_indices = [i for i in item_similarities.argsort()[::-1] if i != index]
    
    return [ (index, item_similarities[index]) for index in related_indices][0:top_n]

### Example 9-10. Item-based collaborative filtering recoomendations: Take 2

In [ ]:
def paper_recommender(items_df, paper_index, top_n):
    if paper_index in model_df.index:
        
        print('Based on the paper:')
        print('Paper index = ', model_df.loc[paper_index].name)
        print('Title :', model_df.loc[paper_index]['title'])
        print('FOS :', model_df.loc[paper_index]['fos'])
        print('Year :', model_df.loc[paper_index]['year'])
        print('Abstract :', model_df.loc[paper_index]['abstract'])
        print('Authors :', model_df.loc[paper_index]['authors'], '\n')
        
        # 요청된 DataFrame 인덱스에 대한 위치 인덱스 정의
        array_ix = model_df.index.get_loc(paper_index)
        
        top_results = piped_collab_filter(items_df, array_ix, top_n)
        
        print("\nTop", top_n, "results: ")
        
        order = 1
        
        for i in range(len(top_results)):
            print(order,'. Paper index = ', model_df.iloc[top_results[i][0]].name)
            print('Similarity score: ', top_results[i][1])
            print('Title :', model_df.iloc[top_results[i][0]]['title'])
            print('FOS :', model_df.iloc[top_results[i][0]]['fos'])
            print('Year :', model_df.iloc[top_results[i][0]]['year'])
            print('Abstract :', model_df.iloc[top_results[i][0]]['abstract'])
            print('Authors :', model_df.iloc[top_results[i][0]]['authors'], '\n')
            if order < top_n: order += 1
        
    else:
        print('Whoops! Choose another paper. Try something from here: \n', model_df.index[100:200])

In [ ]:
paper_recommender(second_feature, 2, 3)

## Example 9-11. Maintaining index assignment during conversions

In [ ]:
model_df.loc[21]

In [ ]:
model_df.iloc[21]

In [ ]:
model_df.index.get_loc(30)

## __세 번째 단계: 추가 피쳐 = 추가 정보__
- 이 실험은 출간 년도와 연구 분야가 유사한 논문을 추천하기에 충분할 것이라는 처음의 가설을 지지하지 못하고 있다. 이 시점에서 몇가지 옵션이 있다.
    - 원본 데이터셋을 좀 더 많이 사용해 더 나은 결과를 얻을 수 있는지 확인해본다.
    - 좋은 추천 결과를 제공할 수 있을 만큼 충분한 데이터인지 좀 더 시간을 들여 탐색한다.
    - 현재 모델에 피처를 더 추가한다.

첫 번째 옵션은 샘플링한 데이터에 문제가 있다고 가정한다. 그것이 사실일 수도 있지만, 이는 더 나은 결과를 얻기 위해 데이터 더미를 휘젓는 것과 비슷하다.

두 번째 옵션은 기본적으로 원본 데이터에 대해 더 많이 이해할 수 있게 해준다. 이 옵션은 탐색 과정에서 피처와 모델 선택에 따른 변경 사항에 대해 지속적으로 재검토돼야 한다. 이 예제에서 선택한 초기 샘플은 이를 반영하고 있다. 데이터셋에서 사용할 수 있는 변수들이 많이 있으믈 이 옵션을 더 이상 진행하지 않는다.

세 번째 옵션을 적용해 피처를 더 추가함으로써 현재 모델을 더 발전시켜보자. 각 항목에 대해 더 많은 정보를 제공하면 유사도 점수를 향상시키고 더 나은 추천 결과를 얻을 수 있다. 초기 탐색을 기초로 다음 단계에서는 가장 많은 정보를 가진 초록(abstract)과 저자(authors)필드에 중점을 둔다

### 학술 논문 추천 시스템: 테이크 3

### Example 9-12. Stopwords + tf-idf
- 불용어 처리 + tf-idf

In [ ]:
# sklearn을 사용하기 위해 NaN 항목을 채워준다
filled_df = model_df.fillna('None')

In [ ]:
filled_df['abstract'].head()

In [ ]:
# abstract: 불용어, 빈도기반 필터링
vectorizer = TfidVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
X_abstract = vectorizer.fit_transform(filled_df['abstract'])

X_abstract

In [ ]:
print("n_samples: %d, n_features: %d" % X_abstract.shape)

In [ ]:
X_yrs.shape[1] + X_fos.shape[1] + X_abstract.shape[1]

In [ ]:
# 10399 x 56139 array

%time third_features = np.append(second_features, X_abstract.toarray(), axis=1)

In [ ]:
paper_recommender(third_feature, 2, 3)

### Example 9-13. One-hot encoding using scikit-learn's DictVectorizer

In [ ]:
authors_df = pd.DataFrame(filled_df.authors)
authors_df.head()

In [ ]:
authors_list = []

for row in authors_df.itertuples():
    # 각 시리즈 인덱스부터 딕셔너리 생성
    if type(row.authors) is str:
        y = {'None': row.Index}
    if type(row.authors) is list:
        # 이 키와 값을 딕셔너리에 추가
        y = dict.fromkeys(row.authors[0].values(), row.Index)
    authors_list.append(y)

In [ ]:
authors_list[0:5]

In [ ]:
v = DicVectorizer(sparse=False)
D = authors_list
X_authors = v.fit_transform(D)

X_authors

In [ ]:
print("n_samples: %d, n_features: %d" % X_authors.shape)

In [ ]:
X_yrs.shape[1] + X_fos.shape[1] + X_abstract.shape[1] + X_authors.shape[1]

In [ ]:
# 10399 x 70167 array

%time fourth_features = np.append(third_features, X_authors, axis=1)

### Example 9-14. Item-based collaborative filtering recommendation: Take 3

In [ ]:
paper_recommender(fourth_features, 2, 3)

## __NEXT STEPS__
- 